# Ozone Layer Detection

In this notebook, we use three methods to analyze the [Ozone Level Detection dataset, obtained from UCI](https://archive.ics.uci.edu/ml/datasets/Ozone+Level+Detection).



*   Support Vector Machine
*   K-Nearest Neighbour
*   Multilayer Perceptron Feed-Forward Network



In [ ]:
# get the files

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/ozone/eighthr.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/ozone/eighthr.names

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/ozone/onehr.data
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/ozone/onehr.names
  
!wget https://github.com/aaakashkumar/Ozone-Level-Detection/raw/master/eighthr.csv

## Imports

It's a good idea to make all the necessary imports in the beginning of the notebook. However, we make imports catering to the specific methods in their respective sections, for clarity.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Data Cleanup

The dataset contains 2536 rows of values, and 73 attributes. Out of these, 687 rows contain missing values represented as "`?`''.

The `eighthr.data` was converted to CSV using [OpenRefine](http://openrefine.org/), and respective column names were added from `eighthr.names` file.

Here we read the csv file and replace the missing values (`?`) by row means.

In [ ]:
df = pd.read_csv('eighthr.csv')
df_original = pd.read_csv('eighthr.csv')

### Examining the dataset
It's a good idea to get to know the data a little bit before working with it.

In [ ]:
print(df.shape)
df.head()

We'll print out a quick summary of a few useful statistics on each column.

This will include things like mean, standard deviation, max, min, and various quantiles.

In [ ]:
df.describe()

We remove the `Date` column, as it is not useful for prediction. 

In [ ]:
df.drop(columns='Date', inplace=True)

In [ ]:
df.head()

### Replace missing values using [Imputer](https://sklearn.org/modules/generated/sklearn.preprocessing.Imputer.html)

In [ ]:
df.replace(to_replace='?', value=np.nan, inplace=True)

In [ ]:
df.head()

We store the target to be predicted in Y

In [ ]:
# Splitting the dataset into feature(X) and target(Y)
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
df = pd.DataFrame(imputer.fit_transform(df), dtype='float64')
df.columns = ['WSR0','WSR1','WSR2','WSR3','WSR4','WSR5','WSR6','WSR7','WSR8','WSR9','WSR10','WSR11','WSR12','WSR13','WSR14','WSR15','WSR16','WSR17','WSR18','WSR19','WSR20','WSR21','WSR22','WSR23','WSR_PK','WSR_AV','T0','T1','T2','T3','T4','T5','T6','T7','T8','T9','T10','T11','T12','T13','T14','T15','T16','T17','T18','T19','T20','T21','T22','T23','T_PK','T_AV','T85','RH85','U85','V85','HT85','T70','RH70','U70','V70','HT70','T50','RH50','U50','V50','HT50','KI','TT','SLP','SLP_','Precp','Result']
df.head()

In [ ]:
# check if NaN values exist
if np.nan in df['T_PK'].values.tolist():
  print("NaN values found")
else:
  print("No NaN values found")

In [ ]:
# Cleaning data using Imputer Class
X = imputer.fit_transform(X)

Now, we split the data into two parts — train and test.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2, random_state=9)

## Data Visualization

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import seaborn as sns

%matplotlib inline

### Peak Temperatures plot

In [ ]:
fig = plt.figure(figsize = (14,4))
title = fig.suptitle("Peak Temperatures", fontsize=14)
fig.subplots_adjust(top=0.85, wspace=0.3)

# Histogram
ax = fig.add_subplot(1,2, 1)
ax.set_xlabel("Temperature")
ax.set_ylabel("Frequency") 
ax.text(1.2, 800, r'$\mu$='+str(round(df['T_PK'].mean(),2)), 
         fontsize=12)
freq, bins, patches = ax.hist(df['T_PK'], color='steelblue', bins=15,
                                    edgecolor='black', linewidth=1)


# Density Plot
ax1 = fig.add_subplot(1,2, 2)
ax1.set_xlabel("Temperature")
ax1.set_ylabel("Frequency") 
sns.kdeplot(df['T_PK'], ax=ax1, shade=True, color='steelblue')

### Peak Wind Speed Plots

In [ ]:
fig = plt.figure(figsize = (14,4))
title = fig.suptitle("Peak WSR", fontsize=14)
fig.subplots_adjust(top=0.85, wspace=0.3)

# Histogram
ax = fig.add_subplot(1,2, 1)
ax.set_xlabel("Wind Speed")
ax.set_ylabel("Frequency") 
ax.text(1.2, 800, r'$\mu$='+str(round(df['WSR_PK'].mean(),2)), 
         fontsize=12)
freq, bins, patches = ax.hist(df['WSR_PK'], color='steelblue', bins=15,
                                    edgecolor='black', linewidth=1)


# Density Plot
ax1 = fig.add_subplot(1,2, 2)
ax1.set_xlabel("Wind Speed")
ax1.set_ylabel("Frequency") 
sns.kdeplot(df['T_PK'], ax=ax1, shade=True, color='steelblue')

### Correlation Matrix

Here we depict the pair-wise correlation matrix as a heatmap, for a few attributes of the dataset

In [ ]:
cormap = df.corr()
fig, ax = plt.subplots(figsize=(50,50))
sns.heatmap(cormap,cmap="YlGnBu", annot = True)

In [ ]:
# Correlation Matrix Heatmap
f, ax = plt.subplots(figsize=(10, 6))
subset_attributes = ['WSR_PK','T_PK','T85','T70','RH70','U70','V70','HT70','KI','TT','SLP','SLP_','Precp']
corr = df[subset_attributes].corr()
hm = sns.heatmap(round(corr,2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
                 linewidths=.05)
f.subplots_adjust(top=0.93)
t= f.suptitle('Ozone Level Attributes Correlation Heatmap', fontsize=14)

### Factorplot

Here we try to establish a relation between peak temperature and the result.

In [ ]:
df['ozone_label'] = df['Result'].apply(lambda value: 'Non Ozone Day' if value == 0 else 'Ozone Day')
df['ozone_label'] = pd.Categorical(df['ozone_label'], categories=['Non Ozone Day', 'Ozone Day'])

In [ ]:
# Visualizing 3-D categorical data using bar plots
# leveraging the concepts of hue and facets
fc = sns.factorplot(x="T_PK", hue="Result", col="ozone_label", 
                    data=df, kind="count")

## Using Support Vector Machine

### Generating Model

In [ ]:
# Import svm model
from sklearn import svm

# Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

# Train the model using the training sets
clf.fit(X_train, Y_train)

# Predict the response for test dataset
Y_pred = clf.predict(X_test)

### Evaluating the Model

In [ ]:
# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(Y_test, Y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(Y_test, Y_pred)
fig, ax = plt.subplots(figsize=(5,5))
sns.heatmap(mat, annot = True)

## Using K Nearest Neighbors

### Generating Model

In [ ]:
# Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier

# Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)

# Train the model using the training sets
knn.fit(X_train, Y_train)

# Predict the response for test dataset
Y_pred = knn.predict(X_test)

### Model Evaluation

In [ ]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(Y_test, Y_pred)
fig, ax = plt.subplots(figsize=(5,5))
sns.heatmap(mat, annot = True)

## Using Deep Neural Network

We use [Keras](https://keras.io/) to implement the Neural Network model.

In [ ]:
# imports
import keras
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization

Before feeding the data into the model, we standardise it. 

The StandardScaler assumes that the data is normally distributed within each feature and will scale them such that the distribution is now centred around 0, with a standard deviation of 1.

The mean and standard deviation are calculated for the feature and then the feature is scaled based on:

\begin{align*}
  \left(\frac{xi–mean(x)}{stdev(x)}\right)
\end{align*}

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test  = sc.transform(X_test)

The following code defines the model. There are 72 nodes in the input layer, followed by 100 and 50 nodes in the hidden layers, both followed by a ReLU activation unit, and then an output layer followed by Sigmoid activation.

In [ ]:
# Initializer that generates tensors with a normal distribution
initializer = keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=42)

# Define model
model = Sequential()
model.add(Dense(100, kernel_initializer=initializer, input_dim=72, activation= "relu"))
model.add(BatchNormalization())
model.add(Dense(50, kernel_initializer=initializer, activation= "relu"))
model.add(BatchNormalization())
model.add(Dense(1, kernel_initializer=initializer, activation= "sigmoid"))

# Print model Summary
model.summary() 

We calculate the Cross Entropy loss for this model, and print out the accuracy. The optimizer used is Adam, with a learning rate of 0.001.

In [ ]:
# Set up the optimizer
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# Compile model
model.compile(loss="binary_crossentropy" , optimizer=adam, metrics=["accuracy"])

In [ ]:
# Fit Model
history = model.fit(X_train, Y_train, validation_split=0.25, epochs=10, batch_size=16, verbose=1)

In [ ]:
from sklearn.metrics import mean_squared_error

pred = model.predict(X_test)
score = np.sqrt(mean_squared_error(Y_test, pred))
print (score)

### Keras Training History Visualization

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
if 'accuracy' in history.history:
    plt.plot(history.history['accuracy'])
if 'val_accuracy' in history.history:
    plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
if 'loss' in history.history:
    plt.plot(history.history['loss'])
if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


## Accuracies

SVM: 0.94%

KNN: 93.68%

DNN: 95.59%